## Import Libraries

In [1]:
from langfuse.langchain import CallbackHandler

In [3]:
from langgraph.prebuilt import create_react_agent

In [1]:
from llama_index.readers.file import PDFReader, PyMuPDFReader
from llama_index.core import Document

# enviroment variables
import os
from dotenv import load_dotenv
load_dotenv()

True

## Load the Data

In [2]:
loader = PyMuPDFReader()
data_path = r"C:\Users\USER\Desktop\Projects\rag_llamaindex\tools\rag_docs\legal_docs\international law handook.pdf"
docs0 = loader.load(file_path=data_path)

In [3]:
print(len(docs0))

681


In [4]:
docs0[:5]

[Document(id_='e46666b5-92b2-4bae-940d-d8d008f15adb', embedding=None, metadata={'total_pages': 681, 'file_path': 'C:\\Users\\USER\\Desktop\\Projects\\rag_llamaindex\\tools\\rag_docs\\legal_docs\\international law handook.pdf', 'source': '1'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='INTERNATIONAL LAW HANDBOOK\nCOLLECTION OF INSTRUMENTS\nBOOK ONE\n', path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}'),
 Document(id_='3ebae237-2f95-4774-a4d3-c0c0411c887b', embedding=None, metadata={'total_pages': 681, 'file_path': 'C:\\Users\\USER\\Desktop\\Projects\\rag_llamaindex\\tools\\rag_docs\\legal_docs\\international law handook.pdf', 'source': '2'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_temp

In [5]:
from llama_index.core import Document

doc_text = "\n\n".join([d.get_content() for d in docs0])
docs = [Document(text=doc_text)]

In [6]:
len(docs)

1

In [7]:
docs[0]

Document(id_='ac02f46c-b6da-4a8d-b517-5a72f610b87c', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='INTERNATIONAL LAW HANDBOOK\nCOLLECTION OF INSTRUMENTS\nBOOK ONE\n\n\nThe photograph on the cover is of a stained \nglass window in the United Nations \nHeadquarters building in New York. The \nstaff of the United Nations and Marc Chagall \ndonated the stained glass panel designed \nby the French artist as a memorial to Dag \nHammarskjöld and 15 others who died in a \nplane crash while on a peace mission in the \nCongo in 1961. Dag Hammarskjöld served as \nthe second Secretary-General of the United \nNations from 10 April 1953 until his death \non 18 September 1961. He introduced the \nconcept of peacekeeping and was awarded \nthe Nobel Peace Prize. He also defined the \nrole of an international civil ser

In [8]:

"""
chunking: output a hierarchy of nodes, 
from top-level nodes with bigger chunk sizes to child nodes with smaller chunk sizes, 
where each child node has a parent node with a bigger chunk size.
"""
from llama_index.core.node_parser import (
    HierarchicalNodeParser,
    SentenceSplitter,
)

node_parser = HierarchicalNodeParser.from_defaults()
nodes = node_parser.get_nodes_from_documents(docs)
len(nodes)

5954

In [9]:
"""
Function for fetching “leaf” nodes within a node list. 
These are nodes that don’t have children of their own.
"""

from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes
leaf_nodes = get_leaf_nodes(nodes)
print(len(leaf_nodes))
root_nodes = get_root_nodes(nodes)

4635


In [14]:
"""
Load into storage weaviate
"""
import weaviate
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import StorageContext

# connect to weaviate
cluster_url = os.getenv("WEAVIATE_URL")
weaviate_api_key = os.getenv("WAEVIATE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

client = weaviate.connect_to_wcs(
    cluster_url=cluster_url,
    auth_credentials=weaviate.auth.AuthApiKey(weaviate_api_key),
)

# create vector store
vector_store = WeaviateVectorStore(
    weaviate_client=client,
    index_name="Document",
)

# Create storage context
storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore(),
    vector_store=vector_store,
)

# Create a VectorStoreIndex and add documents
index = VectorStoreIndex.from_documents(
    nodes,
    storage_context=storage_context
)

print(f"{len(nodes)} documents added to Weaviate!")

ImportError: cannot import name 'VectorStoreIndex' from 'llama_index' (unknown location)

ImportError: cannot import name 'initialize_agent' from 'langchain.agents' (c:\Users\USER\Desktop\Projects\rag_llamaindex\.venv\Lib\site-packages\langchain\agents\__init__.py)